In [6]:
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
%%capture
!pip install transformers peft torch

In [8]:
!pip install bitsandbytes


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 MB 31.1 MB/s eta 0:00:00


In [1]:
!pip install --force-reinstall bitsandbytes

  Using cached bitsandbytes-0.45.4-py3-none-manylinux_2_24_x86_64.whl.metadata (5.0 kB)
  Using cached torch-2.6.0-cp311-cp311-manylinux1_x86_64.whl.metadata (28 kB)
  Using cached numpy-2.2.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached typing_extensions-4.13.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached networkx-3.4.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl.metadata (2.9 kB)
  Using cached fsspec-2025.3.2-py3-none-any.whl.metadata (11 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.4.127-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.1.0.70-py3-none-manylinux2014_x86_64.whl.metadata (1.6 kB)
  Us

In [2]:
!pip install -U bitsandbytes

# load model and emo question csv

In [3]:
%%capture
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 1. load trained model and tokenizer
model_path = "/content/drive/My Drive/LLM_project_health_chatbot"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
print(model.config)

LlamaConfig {
  "_attn_implementation_autoset": true,
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "head_dim": 128,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "mlp_bias": false,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pad_token_id": 128004,
  "pretraining_tp": 1,
  "quantization_config": {
    "_load_in_4bit": true,
    "_load_in_8bit": false,
    "bnb_4bit_compute_dtype": "bfloat16",
    "bnb_4bit_quant_storage": "uint8",
    "bnb_4bit_quant_type": "nf4",
    "bnb_4bit_use_double_quant": true,
    "llm_int8_enable_fp32_cpu_offload": false,
    "llm_int8_has_fp16_weight": false,
    "llm_int8_skip_modules": [
      "lm_head",
      "multi_modal_projector",
      "merger",
      "modality_projection"
    

In [4]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

csv_path = "/content/drive/MyDrive/emo test question 50.csv"
df = pd.read_csv(csv_path)

print(df.head())
print(df.info())

                                        questionText
0  I feel like I would be more comfortable as a g...
1  I started dating a guy I met online. He told m...
2  About 3 years ago or so I was skinny, but I wa...
3  A few nights ago I talked to this girl I know ...
4  She cried because she lost all trust in me. No...
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50 entries, 0 to 49
Data columns (total 1 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   questionText  50 non-null     object
dtypes: object(1)
memory usage: 532.0+ bytes
None


# inference question in csv

In [16]:
def generate_response(input_text, model, tokenizer):
    if not input_text.strip():
        return "Please provide a valid query."

    # Define a custom prompt template
    template = """
    You are a supportive assistant. Provide a single, helpful, and empathetic response to the user's query.
    User query: {user_query}
    Response:
    """
    prompt = template.format(user_query=input_text)

    # Prepare input
    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to("cuda")

    # Generate response
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=250,  # Limit response length
        temperature=0.7,     # Adjust for more focused responses
        top_p=0.9,           # Use nucleus sampling
        repetition_penalty=1.1,
        num_return_sequences=1  # Ensure a single response is generated
    )

    # Decode and return response
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Post-process and validate response
    if "I'm sorry" not in response:  # Example of a simple validation
        return response.strip()
    else:
        return "I couldn't provide a helpful response. Please try again."

# Iterate over each question in the DataFrame
responses = []

for question in df["questionText"]:
    response = generate_response(question, model, tokenizer)
    responses.append(response)

# Add the responses to the DataFrame
df["Response"] = responses

# Save the results to a CSV file
output_path = "/content/drive/MyDrive/emo_eval_answer_50.csv"
df.to_csv(output_path, index=False)

print(f"Saved answers to {output_path}")

Saved answers to /content/drive/MyDrive/emo_eval_answer_50.csv


In [17]:
def generate_response(input_text, model, tokenizer):
    if not input_text.strip():
        return "Please provide a valid query."

    # Define a custom prompt template
    template = """
    You are a supportive assistant. Provide a single, helpful, and empathetic response to the user's query.
    User query: {user_query}
    Response:
    """
    prompt = template.format(user_query=input_text)

    # Prepare input
    inputs = tokenizer(
        prompt,
        return_tensors="pt"
    ).to("cuda")

    # Generate response
    outputs = model.generate(
        input_ids=inputs['input_ids'],
        attention_mask=inputs['attention_mask'],
        max_new_tokens=250,
        temperature=0.7,     # Adjust for more focused responses
        top_p=0.9,           # Use nucleus sampling
        repetition_penalty=1.1,
        num_return_sequences=1  # Ensure a single response is generated
    )

    # Decode and return response
    response = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]

    # Extract the response portion
    response_start = response.find("Response:") + len("Response:")
    clean_response = response[response_start:].strip()

    # Post-process and validate response
    if "I'm sorry" not in clean_response:  # Example of a simple validation
        return clean_response
    else:
        return "I couldn't provide a helpful response. Please try again."

# Iterate over each question in the DataFrame
responses = []

for question in df["questionText"]:
    response = generate_response(question, model, tokenizer)
    responses.append(response)

# Add the responses to the DataFrame
df["Response"] = responses

# Save the results to a CSV file
output_path = "/content/drive/MyDrive/emo_eval_answer_50.csv"
df.to_csv(output_path, index=False)

print(f"Saved answers to {output_path}")

Saved answers to /content/drive/MyDrive/emo_eval_answer_50.csv
